This notebook produces a table of all the people monitoring sites for which we have access to. Currently we have access to training data for Natural England and North Downs Way sites only.

In [ ]:
os.chdir('..')

In [ ]:
%load_ext autoreload
%autoreload 2



from model_config import *
from model_packages import *
from model_utils import *


In [ ]:
# Natural England counter data
NE_counter_locations=assgn_lat_lon(ne_countr_locn_file,'counter')

# North downs way counter data
north_downs_counter_locations=assgn_lat_lon(ndw_countr_locn_file,'counter')

# River and Canal Trust counter data
canal_trst_counter_locations=assgn_lat_lon(crt_countr_locn_file,'Counter')\
[['Counter','latitude','longitude','geometry']]

canal_trst_counter_locations.rename(columns={'Counter':'counter'},inplace=True)

In [ ]:
# collate all counter locations

all_counter_location=pd.concat([NE_counter_locations,north_downs_counter_locations,\
                                canal_trst_counter_locations]).reset_index(drop=True)

In [ ]:
# Change the projection from deg to metres before applying buffer

all_counter_location=all_counter_location.to_crs(crs_mtr)

# Define a 5km buffer buffer zone around every counter site
all_counter_location.geometry=all_counter_location.buffer(bufr_zones_mrts)
all_counter_location['area']=all_counter_location.area/(10**6)
all_counter_location['geom_type']='5km buffer'

In [ ]:
#Keep relevant columns
all_counter_location=all_counter_location[[x for x in all_counter_location.columns if \
                                           x not in ['latitude','longitude']]]

## Plotting Anonymised counter locations

In [ ]:
# anonymise x,y coordinates by selecting random angle and adjusting a given displacement in that direction

NE_counter_locations= anonymise_coordinates(NE_counter_locations, 5000)

north_downs_counter_locations= anonymise_coordinates(north_downs_counter_locations, 5000)

canal_trst_counter_locations= anonymise_coordinates(canal_trst_counter_locations, 5000)

In [ ]:
train_sites=pd.concat([NE_counter_locations, north_downs_counter_locations])
train_sites.geometry=train_sites.geometry.centroid


In [ ]:
color_palette = sns.color_palette("colorblind")
fig, ax = plt.subplots()
# Visualisation of all the counter locations anonymised
NE_counter_locations.plot(ax=ax, color=color_palette[1], marker='+',label='Natural England sites :{}'.\
                             format(NE_counter_locations.shape[0]))
contextily.add_basemap(ax,crs= crs_mtr,source=contextily.providers.Esri.WorldGrayCanvas)

# north_downs_counter_locations.plot(ax=ax,color=color_palette[2], marker='o',label='North Downs Way sites :{}'.\
#                                    format(north_downs_counter_locations.shape[0]))
# contextily.add_basemap(ax,crs= crs_mtr,source=contextily.providers.Esri.WorldGrayCanvas)

# canal_trst_counter_locations.plot(ax=ax,color='green',label='Canal river trust sites :{}'.\
#                                   format(canal_trst_counter_locations.shape[0]))
# contextily.add_basemap(ax,crs= crs_mtr,source=contextily.providers.Esri.WorldGrayCanvas, zoom=10)

ax.legend(loc='best', bbox_to_anchor=(0.45,0.15), frameon=1, facecolor='white')
# frame= ax.legend.get_frame()
ax.axis('off')


pathlib.Path(f"./outputs/").mkdir(parents=True, exist_ok=True)
# fig.savefig(f"./outputs/sites_x_source_NE_NDW.png", format= 'png', dpi=300, bbox_inches='tight')